In [5]:
from crowemi_trades.storage.s3_storage import S3Storage
from datetime import datetime, tzinfo
import polars as pl

stor = S3Storage()
t = stor.read_parquet(bucket="crowemi-trades", key="C:EURUSD/minute/5/2023/01/20230102.parquet.gzip",)
print(t)

shape: (288, 8)
┌──────────┬────────────┬──────────────┬──────────┬───────────────┬────────┬───────────────┬───────┐
│ ticker   ┆ queryCount ┆ resultsCount ┆ adjusted ┆ results       ┆ status ┆ request_id    ┆ count │
│ ---      ┆ ---        ┆ ---          ┆ ---      ┆ ---           ┆ ---    ┆ ---           ┆ ---   │
│ str      ┆ i64        ┆ i64          ┆ bool     ┆ struct[8]     ┆ str    ┆ str           ┆ i64   │
╞══════════╪════════════╪══════════════╪══════════╪═══════════════╪════════╪═══════════════╪═══════╡
│ C:EURUSD ┆ 1430       ┆ 288          ┆ true     ┆ {1.0702,1.070 ┆ OK     ┆ c9ba078d47c9a ┆ 288   │
│          ┆            ┆              ┆          ┆ 27,1.06955,7, ┆        ┆ c140c490cb176 ┆       │
│          ┆            ┆              ┆          ┆ 1.0702...     ┆        ┆ 9367dd        ┆       │
│ C:EURUSD ┆ 1430       ┆ 288          ┆ true     ┆ {1.0703,1.070 ┆ OK     ┆ c9ba078d47c9a ┆ 288   │
│          ┆            ┆              ┆          ┆ 3,1.0702,4,1. ┆        

In [6]:
print(t['results'])

shape: (288,)
Series: 'results' [struct[8]]
[
	{1.0702,1.07027,1.06955,7,1.07027,1672617600000,7,1.07}
	{1.0703,1.0703,1.0702,4,1.07025,1672617900000,4,1.0703}
	{1.0703,1.0704,1.07025,6,1.07025,1672618200000,6,1.0703}
	{1.06963,1.06965,1.0695,8,1.06955,1672618500000,8,1.0696}
	{1.06933,1.06968,1.06933,10,1.06962,1672618800000,10,1.0695}
	{1.06953,1.06977,1.06948,30,1.06965,1672619100000,30,1.0697}
	{1.0699,1.0701,1.06963,53,1.06977,1672619400000,53,1.0698}
	{1.0699,1.0699,1.06926,46,1.0698,1672619700000,46,1.0696}
	{1.07,1.0702,1.06901,56,1.0696,1672620000000,56,1.0698}
	{1.0705,1.0706,1.06973,51,1.0701,1672620300000,51,1.0702}
	{1.0706,1.071,1.0701,41,1.0706,1672620600000,41,1.0707}
	{1.0706,1.0708,1.06979,52,1.07009,1672620900000,52,1.0705}
	...
	{1.0667,1.0668,1.0661,73,1.0665,1672700100000,73,1.0664}
	{1.06797,1.0683,1.06655,391,1.06672,1672700400000,391,1.0675}
	{1.068,1.06831,1.0678,361,1.06799,1672700700000,361,1.0681}
	{1.06637,1.0682,1.06628,678,1.06803,1672701000000,678,1.067

In [11]:
# close, high, low, # txn, open, timestamp, volume, vwap
res = t['results']
l = pl.DataFrame(map(lambda x: { 'c': x['c'], 'h': x['h'], 'l': x['l'], 'n': x['n'], 'o': x['o'], 't': x['t'], 'ts': datetime.utcfromtimestamp(x['t']/1000), 'v': x['v'], 'vw': x['vw'], }, res))
print(l.sort('ts', reverse=True))


print(f"high: {l['h'].max()}")
print(f"low: {l['l'].min()}")

shape: (288, 9)
┌─────────┬─────────┬─────────┬─────┬─────┬───────────────┬─────────────────────┬─────┬────────┐
│ c       ┆ h       ┆ l       ┆ n   ┆ ... ┆ t             ┆ ts                  ┆ v   ┆ vw     │
│ ---     ┆ ---     ┆ ---     ┆ --- ┆     ┆ ---           ┆ ---                 ┆ --- ┆ ---    │
│ f64     ┆ f64     ┆ f64     ┆ i64 ┆     ┆ i64           ┆ datetime[μs]        ┆ i64 ┆ f64    │
╞═════════╪═════════╪═════════╪═════╪═════╪═══════════════╪═════════════════════╪═════╪════════╡
│ 1.06764 ┆ 1.068   ┆ 1.0673  ┆ 776 ┆ ... ┆ 1672703700000 ┆ 2023-01-02 23:55:00 ┆ 776 ┆ 1.0676 │
│ 1.06745 ┆ 1.0678  ┆ 1.0669  ┆ 564 ┆ ... ┆ 1672703400000 ┆ 2023-01-02 23:50:00 ┆ 564 ┆ 1.0674 │
│ 1.06711 ┆ 1.0677  ┆ 1.0667  ┆ 598 ┆ ... ┆ 1672703100000 ┆ 2023-01-02 23:45:00 ┆ 598 ┆ 1.067  │
│ 1.06708 ┆ 1.068   ┆ 1.06687 ┆ 528 ┆ ... ┆ 1672702800000 ┆ 2023-01-02 23:40:00 ┆ 528 ┆ 1.0671 │
│ ...     ┆ ...     ┆ ...     ┆ ... ┆ ... ┆ ...           ┆ ...                 ┆ ... ┆ ...    │
│ 1.06963 ┆ 1.

In [12]:
import plotly.graph_objects as go
import plotly.express as px

fig = go.Figure(
    data=[
        go.Candlestick(
            x=l['ts'],
            open=l['o'],
            high=l['h'],
            low=l['l'],
            close=l['c'])
    ]
)
fig.show()
px.line(
    l.to_pandas(),
    x='ts',
    y=['c', 'h', 'o', 'l']
)

In [3]:
from crowemi_trades.indicators.session_indicator import SessionIndicator

SessionIndicator.get_session(l)

ModuleNotFoundError: No module named 'indicators'